In [7]:
from openpyxl import load_workbook
from openpyxl.styles import Font
import shutil

total_possible_answers = 729 #81 x 9

**jaccard index calculation**

In [ ]:
from openpyxl import load_workbook
from openpyxl.styles import Font

def highlight_matches_and_jaccard(file_path, sheet_name):
    wb = load_workbook(file_path)
    ws = wb[sheet_name]

    # New sheet name: "Jaccard - SheetName"
    new_sheet_name = f'Jaccard - {sheet_name}'
    if new_sheet_name in wb.sheetnames:
        del wb[new_sheet_name]
    new_sheet = wb.create_sheet(title=new_sheet_name)

    new_row_index = 1
    all_jaccards = []  # Store per-cell Jaccard indices

    for row in ws.iter_rows():
        for cell in row:
            if cell.value:
                parts = cell.value.split('|')
                if len(parts) != 2:
                    continue

                values_table1 = {v.strip() for v in parts[0].split(',') if v.strip()}
                values_table2 = {v.strip() for v in parts[1].split(',') if v.strip()}

                black_matches = values_table1 & values_table2
                red_misses = values_table1 - values_table2
                blue_extras = values_table2 - values_table1

                count_black = len(black_matches)
                count_red = len(red_misses)
                count_blue = len(blue_extras)

                denom = count_black + count_red + count_blue
                jaccard = round((count_black * 100 / denom), 2) if denom > 0 else 100.0
                all_jaccards.append(jaccard)

                # Write cell-by-cell values with colors
                new_col_index = 1
                for val in values_table1:
                    color = "000000" if val in values_table2 else "FF0000"
                    new_cell = new_sheet.cell(row=new_row_index, column=new_col_index, value=val)
                    new_cell.font = Font(color=color)
                    new_col_index += 1

                for val in blue_extras:
                    new_cell = new_sheet.cell(row=new_row_index, column=new_col_index, value=val)
                    new_cell.font = Font(color="0000FF")
                    new_col_index += 1

                # Write Jaccard value at end of row
                new_cell = new_sheet.cell(row=new_row_index, column=new_col_index, value=f"Jaccard: {jaccard}%")
                new_cell.font = Font(color="008000", bold=True)
                new_row_index += 1

    # Write average Jaccard at the bottom
    if all_jaccards:
        avg_jaccard = round(sum(all_jaccards) / len(all_jaccards), 2)
        summary_cell = new_sheet.cell(row=new_row_index + 2, column=1, value=f"Average Jaccard Index: {avg_jaccard}%")
        summary_cell.font = Font(color="000080", bold=True)
    else:
        avg_jaccard = 0.0

    wb.save(file_path)
    print(f"Sheet '{sheet_name}' processed. Avg Jaccard: {avg_jaccard}%")
    return avg_jaccard

In [ ]:
import pandas as pd

# Define your function
def process_sheet(file_path, sheet_name, i):
    print(f"Processing sheet: {sheet_name}")
    new_file_path = "/kaggle/working/jaccard_results_"+str(i)+".xlsx"
    shutil.copy(file_path, new_file_path)
    acc = highlight_matches_and_jaccard(new_file_path, sheet_name)
    return(acc)
    
# Load the Excel file
excel_path = "/kaggle/input/grok-3-mini-def-all-9-results/grok-3-mini_def_all_9_results.xlsx"
excel_file = pd.ExcelFile(excel_path)

i=1
overall_jaccards = []  
# Iterate over all sheet names and call your function
for sheet_name in excel_file.sheet_names:
    avg_jaccard = process_sheet(excel_path, sheet_name, i)
    overall_jaccards.append(avg_jaccard)
    i=i+1

overall_avg = round(sum(overall_jaccards) / len(overall_jaccards), 2)
print(f"Overall Average Jaccard Index: {overall_avg}%")

**Prediction interval calculation**

In [ ]:
import numpy as np
import scipy.stats as stats

def prediction_interval(samples, confidence=0.95):
    mean = np.mean(samples)
    std_dev = np.std(samples, ddof=1)  # sample standard deviation
    n = len(samples)

    t_crit = stats.t.ppf((1 + confidence) / 2, df=n - 1)
    margin_of_error = t_crit * std_dev * np.sqrt(2 / n)

    lower_bound = mean - margin_of_error
    upper_bound = mean + margin_of_error

    return lower_bound, upper_bound

In [ ]:
import numpy as np

# Example sample data
samples = [12.5, 13.0, 12.8, 13.2, 12.9]

# Call the function
lower, upper = prediction_interval(samples)

# Print the result
print(f"Prediction interval: ({lower:.2f}, {upper:.2f})")

**HEATMAPS**

**OUTPUT FOR HEATMAP**

In [8]:
from openpyxl import load_workbook

def jaccard_per_relation_for_heatmap(file_path, sheet_name):
    wb = load_workbook(file_path)
    ws = wb[sheet_name]

    new_sheet_name = f'Jaccard - {sheet_name}'
    if new_sheet_name in wb.sheetnames:
        del wb[new_sheet_name]
    new_sheet = wb.create_sheet(title=new_sheet_name)

    new_row_index = 1
    all_jaccards = []

    for row in ws.iter_rows():
        for cell in row:
            if cell.value:
                parts = cell.value.split('|')
                if len(parts) != 2:
                    continue

                values_table1 = {v.strip() for v in parts[0].split(',') if v.strip()}
                values_table2 = {v.strip() for v in parts[1].split(',') if v.strip()}

                black_matches = values_table1 & values_table2
                red_misses = values_table1 - values_table2
                blue_extras = values_table2 - values_table1

                count_black = len(black_matches)
                count_red = len(red_misses)
                count_blue = len(blue_extras)

                denom = count_black + count_red + count_blue
                jaccard = round((count_black * 100 / denom), 2) if denom > 0 else 100.0
                jaccard_decimal = round(jaccard / 100, 4)
                all_jaccards.append(jaccard_decimal)

                # Write only Jaccard value as decimal
                new_sheet.cell(row=new_row_index, column=1, value=jaccard_decimal)
                new_row_index += 1

    # Write average Jaccard at the bottom
    if all_jaccards:
        avg_jaccard = round(sum(all_jaccards) / len(all_jaccards), 4)
        new_sheet.cell(row=new_row_index + 2, column=1, value=avg_jaccard)
    else:
        avg_jaccard = 0.0

    wb.save(file_path)
    print(f"Sheet '{sheet_name}' processed. Avg Jaccard: {avg_jaccard}")
    return avg_jaccard

In [6]:
import pandas as pd

# Define your function
def process_sheet(file_path, sheet_name, i):
    print(f"Processing sheet: {sheet_name}")
    new_file_path = "/kaggle/working/jaccard_results_"+str(i)+".xlsx"
    shutil.copy(file_path, new_file_path)
    acc = jaccard_per_relation_for_heatmap(new_file_path, sheet_name)
    return(acc)
    
# Load the Excel file
excel_path = "/kaggle/input/grok-3-mini-def-all-3-results/grok-3-mini_def_all_3_results.xlsx"
excel_file = pd.ExcelFile(excel_path)

i=1
overall_jaccards = []  
# Iterate over all sheet names and call your function
for sheet_name in excel_file.sheet_names:
    avg_jaccard = process_sheet(excel_path, sheet_name, i)
    overall_jaccards.append(avg_jaccard)
    i=i+1

overall_avg = round(sum(overall_jaccards) / len(overall_jaccards), 2)
print(f"Overall Average Jaccard Index: {overall_avg}%")

Processing sheet: Φύλλο1
Sheet 'Φύλλο1' processed. Avg Jaccard: 0.8899
Processing sheet: Φύλλο2
Sheet 'Φύλλο2' processed. Avg Jaccard: 0.8875
Processing sheet: Φύλλο3
Sheet 'Φύλλο3' processed. Avg Jaccard: 0.8964
Overall Average Jaccard Index: 0.89%


In [ ]:
pip install pandas seaborn openpyxl matplotlib

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load Excel file
file_path = "/kaggle/input/heatmap-per-model/heatmap_per_model.xlsx"

# Shared axis labels and annotations
labels_df_full = pd.read_excel(file_path, sheet_name='Labels', header=None)
x_labels = labels_df_full.iloc[0, 1:].tolist()
y_labels = labels_df_full.iloc[1:, 0].tolist()
labels_df = labels_df_full.iloc[1:, 1:]
labels_df.index = y_labels
labels_df.columns = x_labels

# Function to load value data from a sheet
def load_values(sheet_name):
    df = pd.read_excel(file_path, sheet_name=sheet_name, header=None).iloc[1:, 1:]
    df.index = y_labels
    df.columns = x_labels
    return df.apply(pd.to_numeric, errors='coerce')

# Load all 4 value sheets
values_df_1 = load_values('o3-default-values')
values_df_2 = load_values('gemini-2.5-flash-default')
values_df_3 = load_values('claude-3.7-sonnet-default')
values_df_4 = load_values('grok-3-mini-default')

# Set up 2x2 subplot grid
fig, axes = plt.subplots(2, 2, figsize=(24, 12))

# Titles for each heatmap
titles = [
    'o3 - default',
    'gemini-2.5-flash - default',
    'claude-3.7-sonnet - default',
    'grok-3-mini - default'
]

# Corresponding dataframes
values_list = [values_df_1, values_df_2, values_df_3, values_df_4]

# Generate each heatmap
for i, ax in enumerate(axes.flat):
    sns.heatmap(
        values_list[i],
        annot=labels_df,
        fmt='',
        cmap='viridis',
        cbar=True,
        linewidths=0.2,
        linecolor='grey',
        vmin=0,
        vmax=1,
        annot_kws={"size": 7},
        ax=ax
    )
    ax.set_title(titles[i], fontsize=14, pad=20)
    ax.xaxis.set_label_position('top')
    ax.xaxis.tick_top()

    # Add vertical label on left side of each heatmap
    ax.set_title(titles[i], fontsize=16, pad=25)

    # ax.text(-0.8, values_list[i].shape[0] / 2, titles[i].split(":")[1].strip(),
    #         rotation=90, fontsize=12, va='center', ha='center')

    # Add custom top-left label
    ax.text(-0.55, -0.55, 'R_1/R_2', ha='left', va='top',
            fontsize=10, color='black', backgroundcolor='white')

plt.tight_layout()
plt.show()

**COUNTS PER RELATION**

In [ ]:
from openpyxl import load_workbook
from openpyxl.styles import Font
import shutil

def highlight_matches(file_path, sheet_names):
    wb = load_workbook(file_path)

    # Create new summary sheet
    new_sheet = wb.create_sheet(title='Highlighted Values')
    new_row_index = 1  # Row index in new sheet

    # Initialize all counts
    directions = ['N', 'W', 'E', 'S', 'NW', 'SW', 'SE', 'NE', 'B']
    count_black = {d: 0 for d in directions}
    count_red = {d: 0 for d in directions}
    count_blue = {d: 0 for d in directions}

    for sheet_name in sheet_names:
        ws = wb[sheet_name]

        for row in ws.iter_rows():
            for cell in row:
                if cell.value:
                    parts = cell.value.split('|')
                    if len(parts) != 2:
                        continue
                    
                    # Parse predicted and actual values
                    values_table1 = {v.strip() for v in parts[0].split(',')}
                    values_table2 = {v.strip() for v in parts[1].split(',')}

                    new_col_index = 1
                    for val in values_table1:
                        new_cell = new_sheet.cell(row=new_row_index, column=new_col_index, value=val)
                        if val in values_table2:
                            color = "000000"  # Black
                            if val in directions:
                                count_black[val] += 1
                        else:
                            color = "FF0000"  # Red
                            if val in directions:
                                count_red[val] += 1
                        new_cell.font = Font(color=color)
                        new_col_index += 1
                    
                    # Extra values in actual (not in predicted)
                    for val in values_table2 - values_table1:
                        new_cell = new_sheet.cell(row=new_row_index, column=new_col_index, value=val)
                        new_cell.font = Font(color="0000FF")  # Blue
                        if val in directions:
                            count_blue[val] += 1
                        new_col_index += 1
                    
                    new_row_index += 1

    wb.save(file_path)

    print("Aggregated comparison across all sheets complete.")
    print("====== MATCHING (BLACK) ======")
    for d in directions:
        print(f"Total Correctly {d} Predicted - Black (Matching): {count_black[d]}")

    print("\n====== INCORRECT (RED) ======")
    for d in directions:
        print(f"Total Incorrectly {d} Predicted - Red: {count_red[d]}")

    print("\n====== MISSING (BLUE) ======")
    for d in directions:
        print(f"Total Missing {d} Predicted - Blue: {count_blue[d]}")

# ---- Example usage ----
new_file_path = "/kaggle/working/experiment_copy.xlsx"
shutil.copy("/kaggle/input/grok-3-mini-def-all-3-results/grok-3-mini_def_all_3_results.xlsx", new_file_path)

# Call function for all 3 sheets
highlight_matches(new_file_path, ["Φύλλο1", "Φύλλο2", "Φύλλο3"])


**Old diagram output with counts and percentages - deprecated**

In [ ]:
# import pandas as pd
# import matplotlib.pyplot as plt
# import numpy as np

# # Load Excel with first column as row labels (index)
# df = pd.read_excel("/kaggle/input/bar-chart-per-model-per-cd/bar_chart_per_model_per_CD.xlsx", sheet_name="o3-default", index_col=0)

# n_rows = df.shape[0]
# n_cols = df.shape[1]
# bar_width = 0.2
# x = np.arange(n_rows)

# # Calculate percentage per row for stacked bar
# row_sums = df.sum(axis=1)
# df_percent = df.div(row_sums, axis=0) * 100  # percentage values

# # Define custom colors: dark blue, light blue, dark green, light green
# colors = ['#23914f', '#8eedb4', '#26627a', '#68b4d4']

# fig, axs = plt.subplots(1, 2, figsize=(16, 6))

# # === Plot 1: Grouped bar chart ===
# # for i, (col, color) in enumerate(zip(df.columns, colors)):
# #     axs[0].bar(x + i * bar_width, df[col], width=bar_width, label=col, color=color)
# bars_grouped = []
# for i, (col, color) in enumerate(zip(df.columns, colors)):
#     bars = axs[0].bar(x + i * bar_width, df[col], width=bar_width, label=col, color=color)
#     axs[0].bar_label(bars, padding=3, fontsize=10)  # Show bar value
#     bars_grouped.append(bars)

# axs[0].set_xticks(x + bar_width * (n_cols - 1) / 2)
# axs[0].set_xticklabels(df.index, rotation=45, fontsize=12)
# axs[0].set_ylabel("o3-default", fontsize=18)
# axs[0].tick_params(axis='y', labelsize=12)  # smaller y-axis values

# #axs[0].legend(title="Columns", bbox_to_anchor=(0.5, -0.15), loc='upper center', ncol=n_cols)

# # === Plot 2: Stacked bar chart with percentages ===
# bottom = np.zeros(n_rows)
# for col, color in zip(df.columns, colors):
#     axs[1].bar(x, df_percent[col], bottom=bottom, label=col, color=color)
#     bottom += df_percent[col].values

# axs[1].set_xticks(x)
# axs[1].set_xticklabels(df.index, rotation=45, fontsize=12)
# axs[1].tick_params(axis='y', labelsize=12)  # smaller y-axis values

# #axs[1].legend(title="Columns", bbox_to_anchor=(0.5, -0.15), loc='upper center', ncol=n_cols)

# # Create one legend for both plots
# handles, labels = axs[0].get_legend_handles_labels()
# fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, -0.05), ncol=n_cols, prop={'size': 14}, title_fontsize='18'  )        

# plt.tight_layout()  # leave space at bottom for legend
# plt.show()

**GROUPED BAR CHART**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# === Load data from 4 sheets ===
df1 = pd.read_excel("/kaggle/input/bar-chart-per-model-per-cd/bar_chart_per_model_per_CD.xlsx", sheet_name="o3-default", index_col=0)
df2 = pd.read_excel("/kaggle/input/bar-chart-per-model-per-cd/bar_chart_per_model_per_CD.xlsx", sheet_name="gemini-default", index_col=0)
df3 = pd.read_excel("/kaggle/input/bar-chart-per-model-per-cd/bar_chart_per_model_per_CD.xlsx", sheet_name="claude-default", index_col=0)
df4 = pd.read_excel("/kaggle/input/bar-chart-per-model-per-cd/bar_chart_per_model_per_CD.xlsx", sheet_name="grok-3-mini-default", index_col=0)

# === Setup ===
colors = ['#23914f', '#8eedb4', '#26627a', '#68b4d4']
bar_width = 0.2

# Create 2x2 grid of subplots
fig, axs = plt.subplots(2, 2, figsize=(18, 12))
sheet_data = [
    (df1, axs[0, 0], "o3 - default"),
    (df2, axs[0, 1], "gemini-2.5-flash - default"),
    (df3, axs[1, 0], "claude-3.7-sonnet - default"),
    (df4, axs[1, 1], "grok-3-mini - default")
]

# === Function to plot grouped bar chart ===
def plot_grouped_bar(df, ax, title):
    x = np.arange(df.shape[0])
    for i, (col, color) in enumerate(zip(df.columns, colors)):
        bars = ax.bar(x + i * bar_width, df[col], width=bar_width, label=col, color=color)
        ax.bar_label(bars, padding=3, fontsize=10)
    ax.set_xticks(x + bar_width * (df.shape[1] - 1) / 2)
    ax.set_xticklabels(df.index, rotation=45, fontsize=10)
    ax.tick_params(axis='y', labelsize=12)
    ax.set_title(title, fontsize=14)
    #ax.set_ylabel(title, fontsize=18, rotation=90, labelpad=10)  # <- vertical title on left

# === Plot all four charts ===
for df, ax, title in sheet_data:
    plot_grouped_bar(df, ax, title)

# === Shared legend at the bottom ===
handles, labels = axs[0, 0].get_legend_handles_labels()
fig.legend(handles, labels, loc='lower center', bbox_to_anchor=(0.5, -0.02), ncol=len(colors), prop={'size': 14}, title_fontsize='18')

plt.tight_layout(rect=[0, 0.04, 1, 1])  # Adjust for legend
plt.show()


**SCATTER PLOT DIAGRAM**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the Excel file
df1 = pd.read_excel("/kaggle/input/all-accuracies-scatter-plot-v5/all_accuracies_v2.xlsx", sheet_name="gemini-2.5-flash - default", index_col=0)

# Plot
#plt.figure(figsize=(8, 5))
fig, axs = plt.subplots(8, 2, figsize=(14, 16))

x_vals = [1, 2, 3]

# Plot the remaining rows as blue dots only
for label in df1.index:
    y_vals = df1.loc[label].values  # row values
    axs[0, 0].plot(x_vals, y_vals, marker='o', color='#0080ff')


axs[0, 0].set_xticks([1, 2, 3])
axs[0, 0].set_ylim(80, 105)

axs[0, 0].text(
    x=-0.15,
    y=0.5,
    s='gemini-2.5-flash',
    rotation=90,
    fontsize=12,
    va='center',
    ha='center',
    transform=axs[0, 0].transAxes
)

axs[0, 0].text(
    0.95, 0.95, '93.83 ± 2.56, n=3', fontsize=12, ha='right', va='top', transform=axs[0, 0].transAxes
)

axs[0, 0].set_title("default")
axs[0, 0].grid(True)

####################################

# Load the Excel file
df2 = pd.read_excel("/kaggle/input/all-accuracies-scatter-plot-v5/all_accuracies_v2.xlsx", sheet_name="gemini-2.5-flash - set", index_col=0)

# Plot
#plt.figure(figsize=(8, 5))

x_vals = [1, 2, 3]

# Plot the remaining rows as blue dots only
for label in df2.index:
    y_vals = df2.loc[label].values  # row values
    axs[0, 1].plot(x_vals, y_vals, marker='o', color='#0080ff')


axs[0, 1].set_xticks([1, 2, 3])
axs[0, 1].set_ylim(80, 105)


axs[0, 1].text(
    0.95, 0.95, '86.69 ± 0.0, n=3', fontsize=12, ha='right', va='top', transform=axs[0, 1].transAxes
)

axs[0, 1].set_title("fixed seed")
axs[0, 1].grid(True)

#####################################2nd line

# Load the Excel file
df3 = pd.read_excel("/kaggle/input/all-accuracies-scatter-plot-v5/all_accuracies_v2.xlsx", sheet_name="deepseek-r1-0324 - default", index_col=0)

# Plot the remaining rows as blue dots only
for label in df3.index:
    y_vals = df3.loc[label].values  # row values
    axs[1, 0].plot(x_vals, y_vals, marker='o', color='#0080ff')


axs[1, 0].set_xticks([1, 2, 3])
axs[1, 0].set_ylim(80, 105)

axs[1, 0].text(
    x=-0.15,
    y=0.5,
    s='deepseek-r1-0324',
    rotation=90,
    fontsize=12,
    va='center',
    ha='center',
    transform=axs[1, 0].transAxes
)

axs[1, 0].text(
    0.95, 0.95, '92.98 ± 7.55, n=3', fontsize=12, ha='right', va='top', transform=axs[1, 0].transAxes
)

axs[1, 0].set_title("default")
axs[1, 0].grid(True)

####################################

# Load the Excel file
df4 = pd.read_excel("/kaggle/input/all-accuracies-scatter-plot-v5/all_accuracies_v2.xlsx", sheet_name="deepseek-r1-0324 - set", index_col=0)

# Plot the remaining rows as blue dots only
for label in df4.index:
    y_vals = df4.loc[label].values  # row values
    axs[1, 1].plot(x_vals, y_vals, marker='o', color='#0080ff')


axs[1, 1].set_xticks([1, 2, 3])
axs[1, 1].set_ylim(80, 105)


axs[1, 1].text(
    0.95, 0.95, '94.27 ± 5.73, n=3', fontsize=12, ha='right', va='top', transform=axs[1, 1].transAxes
)

axs[1, 1].set_title("fixed seed")
axs[1, 1].grid(True)

#####################################3d line

# Load the Excel file
df5 = pd.read_excel("/kaggle/input/all-accuracies-scatter-plot-v5/all_accuracies_v2.xlsx", sheet_name="o3-default", index_col=0)

# Plot the remaining rows as blue dots only
for label in df5.index:
    y_vals = df5.loc[label].values  # row values
    axs[2, 0].plot(x_vals, y_vals, marker='o', color='#0080ff')


axs[2, 0].set_xticks([1, 2, 3])
axs[2, 0].set_ylim(80, 105)

axs[2, 0].text(
    x=-0.15,
    y=0.5,
    s='o3',
    rotation=90,
    fontsize=12,
    va='center',
    ha='center',
    transform=axs[2, 0].transAxes
)

axs[2, 0].text(
    0.95, 0.95, '99.66 ± 1.12, n=3', fontsize=12, ha='right', va='top', transform=axs[2, 0].transAxes
)

axs[2, 0].set_title("default")
axs[2, 0].grid(True)

####################################

# Load the Excel file
df6 = pd.read_excel("/kaggle/input/all-accuracies-scatter-plot-v5/all_accuracies_v2.xlsx", sheet_name="o3-set", index_col=0)

# Plot the remaining rows as blue dots only
for label in df6.index:
    y_vals = df6.loc[label].values  # row values
    axs[2, 1].plot(x_vals, y_vals, marker='o', color='#0080ff')


axs[2, 1].set_xticks([1, 2, 3])
axs[2, 1].set_ylim(80, 105)


axs[2, 1].text(
    0.95, 0.95, '99.39 ± 3.18, n=2', fontsize=12, ha='right', va='top', transform=axs[2, 1].transAxes
)

axs[2, 1].set_title("fixed seed")
axs[2, 1].grid(True)

#####################################4th line

# Load the Excel file
df6 = pd.read_excel("/kaggle/input/all-accuracies-scatter-plot-v5/all_accuracies_v2.xlsx", sheet_name="o1-default", index_col=0)

# Plot the remaining rows as blue dots only
for label in df6.index:
    y_vals = df6.loc[label].values  # row values
    axs[3, 0].plot(x_vals, y_vals, marker='o', color='#0080ff')


axs[3, 0].set_xticks([1, 2, 3])
axs[3, 0].set_ylim(80, 105)

axs[3, 0].text(
    x=-0.15,
    y=0.5,
    s='o1',
    rotation=90,
    fontsize=12,
    va='center',
    ha='center',
    transform=axs[3, 0].transAxes
)

axs[3, 0].text(
    0.95, 0.95, '97.30 ± 4.39, n=3', fontsize=12, ha='right', va='top', transform=axs[3, 0].transAxes
)

axs[3, 0].set_title("default")
axs[3, 0].grid(True)

####################################

# Load the Excel file
df7 = pd.read_excel("/kaggle/input/all-accuracies-scatter-plot-v5/all_accuracies_v2.xlsx", sheet_name="o1-set", index_col=0)

# Plot the remaining rows as blue dots only
for label in df7.index:
    y_vals = df7.loc[label].values  # row values
    axs[3, 1].plot(x_vals, y_vals, marker='o', color='#0080ff')


axs[3, 1].set_xticks([1, 2, 3])
axs[3, 1].set_ylim(70, 105)


axs[3, 1].text(
    0.95, 0.95, '95.78 ± 20.84, n=2', fontsize=12, ha='right', va='top', transform=axs[3, 1].transAxes
)

axs[3, 1].set_title("fixed seed")
axs[3, 1].grid(True)

#####################################5th line

# Load the Excel file
df7 = pd.read_excel("/kaggle/input/all-accuracies-scatter-plot-v5/all_accuracies_v2.xlsx", sheet_name="o4-mini-default", index_col=0)

# Plot the remaining rows as blue dots only
for label in df7.index:
    y_vals = df7.loc[label].values  # row values
    axs[4, 0].plot(x_vals, y_vals, marker='o', color='#0080ff')


axs[4, 0].set_xticks([1, 2, 3])
axs[4, 0].set_ylim(80, 105)

axs[4, 0].text(
    x=-0.15,
    y=0.5,
    s='o4-mini',
    rotation=90,
    fontsize=12,
    va='center',
    ha='center',
    transform=axs[4, 0].transAxes
)

axs[4, 0].text(
    0.95, 0.95, '94.8 ± 7.24, n=3', fontsize=12, ha='right', va='top', transform=axs[4, 0].transAxes
)

axs[4, 0].set_title("default")
axs[4, 0].grid(True)

####################################

# Load the Excel file
df8 = pd.read_excel("/kaggle/input/all-accuracies-scatter-plot-v5/all_accuracies_v2.xlsx", sheet_name="o4-mini-set", index_col=0)

# Plot the remaining rows as blue dots only
for label in df8.index:
    y_vals = df8.loc[label].values  # row values
    axs[4, 1].plot(x_vals, y_vals, marker='o', color='#0080ff')


axs[4, 1].set_xticks([1, 2, 3])
axs[4, 1].set_ylim(80, 105)


axs[4, 1].text(
    0.95, 0.95, '95.47 ± 0.0, n=3', fontsize=12, ha='right', va='top', transform=axs[4, 1].transAxes
)

axs[4, 1].set_title("fixed seed")
axs[4, 1].grid(True)


#####################################6th line

# Load the Excel file
df9 = pd.read_excel("/kaggle/input/all-accuracies-scatter-plot-v5/all_accuracies_v2.xlsx", sheet_name="grok-3-mini-default", index_col=0)

# Plot the remaining rows as blue dots only
for label in df9.index:
    y_vals = df9.loc[label].values  # row values
    axs[5, 0].plot(x_vals, y_vals, marker='o', color='#0080ff')


axs[5, 0].set_xticks([1, 2, 3])
axs[5, 0].set_ylim(80, 105)

axs[5, 0].text(
    x=-0.15,
    y=0.5,
    s='grok-3-mini',
    rotation=90,
    fontsize=12,
    va='center',
    ha='center',
    transform=axs[5, 0].transAxes
)

axs[5, 0].text(
    0.95, 0.95, '89.13 ± 1.62, n=3', fontsize=12, ha='right', va='top', transform=axs[5, 0].transAxes
)

axs[5, 0].set_title("default")
axs[5, 0].grid(True)

####################################

# Load the Excel file
df10 = pd.read_excel("/kaggle/input/all-accuracies-scatter-plot-v5/all_accuracies_v2.xlsx", sheet_name="grok-3-mini-set", index_col=0)

# Plot the remaining rows as blue dots only
for label in df10.index:
    y_vals = df10.loc[label].values  # row values
    axs[5, 1].plot(x_vals, y_vals, marker='o', color='#0080ff')


axs[5, 1].set_xticks([1, 2, 3])
axs[5, 1].set_ylim(70, 105)


axs[5, 1].text(
    0.95, 0.95, '90.26 ± 5.66, n=3', fontsize=12, ha='right', va='top', transform=axs[5, 1].transAxes
)

axs[5, 1].set_title("fixed seed")
axs[5, 1].grid(True)

#####################################7th line

# Load the Excel file
df11 = pd.read_excel("/kaggle/input/all-accuracies-scatter-plot-v5/all_accuracies_v2.xlsx", sheet_name="claude-3.7-sonnet-default", index_col=0)

# Plot the remaining rows as blue dots only
for label in df11.index:
    y_vals = df11.loc[label].values  # row values
    axs[6, 0].plot(x_vals, y_vals, marker='o', color='#0080ff')


axs[6, 0].set_xticks([1, 2, 3])
axs[6, 0].set_ylim(80, 105)

axs[6, 0].text(
    x=-0.15,
    y=0.5,
    s='claude-3.7-sonnet',
    rotation=90,
    fontsize=12,
    va='center',
    ha='center',
    transform=axs[6, 0].transAxes
)

axs[6, 0].text(
    0.95, 0.95, '90.4 ± 4.32, n=3', fontsize=12, ha='right', va='top', transform=axs[6, 0].transAxes
)

axs[6, 0].set_title("default")
axs[6, 0].grid(True)

####################################

# Load the Excel file
df12 = pd.read_excel("/kaggle/input/all-accuracies-scatter-plot-v5/all_accuracies_v2.xlsx", sheet_name="claude-3.7-sonnet-set", index_col=0)

# Plot the remaining rows as blue dots only
for label in df12.index:
    y_vals = df12.loc[label].values  # row values
    axs[6, 1].plot(x_vals, y_vals, marker='o', color='#0080ff')


axs[6, 1].set_xticks([1, 2, 3])
axs[6, 1].set_ylim(50, 105)


axs[6, 1].text(
    0.95, 0.95, '59.96 ± 0.0, n=3', fontsize=12, ha='right', va='top', transform=axs[6, 1].transAxes
)

axs[6, 1].set_title("fixed seed")
axs[6, 1].grid(True)


#####################################8th line

# Load the Excel file
df12 = pd.read_excel("/kaggle/input/all-accuracies-scatter-plot-v5/all_accuracies_v2.xlsx", sheet_name="deepseek-r1-0528-default", index_col=0)

# Plot the remaining rows as blue dots only
for label in df12.index:
    y_vals = df12.loc[label].values  # row values
    axs[7, 0].plot(x_vals, y_vals, marker='o', color='#0080ff')


axs[7, 0].set_xticks([1, 2, 3])
axs[7, 0].set_ylim(80, 105)

axs[7, 0].text(
    x=-0.15,
    y=0.5,
    s='deepseek-r1-0528',
    rotation=90,
    fontsize=12,
    va='center',
    ha='center',
    transform=axs[7, 0].transAxes
)

axs[7, 0].text(
    0.95, 0.95, '95.03 ± 5.06, n=3', fontsize=12, ha='right', va='top', transform=axs[7, 0].transAxes
)

axs[7, 0].set_title("default")
axs[7, 0].grid(True)

####################################

# Load the Excel file
df12 = pd.read_excel("/kaggle/input/all-accuracies-scatter-plot-v5/all_accuracies_v2.xlsx", sheet_name="deepseek-r1-0528-set", index_col=0)

# Plot the remaining rows as blue dots only
for label in df12.index:
    y_vals = df12.loc[label].values  # row values
    axs[7, 1].plot(x_vals, y_vals, marker='o', color='#0080ff')


axs[7, 1].set_xticks([1, 2, 3])
axs[7, 1].set_ylim(50, 105)


axs[7, 1].text(
    0.95, 0.95, '95.03 ± 9.56, n=3', fontsize=12, ha='right', va='top', transform=axs[7, 1].transAxes
)

axs[7, 1].set_title("fixed seed")
axs[7, 1].grid(True)

fig.align_ylabels()
plt.tight_layout()
plt.show()

**AVERAGE TIME**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load Excel file and select a sheet
file_path = '/kaggle/input/all-accuracies-scatter-plot-v5/all_accuracies_v2.xlsx'
sheet_name = 'average-time'  # Change to your actual sheet name
df = pd.read_excel(file_path, sheet_name=sheet_name, header=None)

# Display data to confirm structure
#print(df)

# Extract x and y from the correct columns
x = df.iloc[:, 0]  # First column for labels
y = df.iloc[:, 1]  # Second column for values

# Create bar chart
plt.figure(figsize=(14, 8))
plt.bar(x, y, color='#0080ff')
#plt.xlabel('Label')
#plt.ylabel('Value')
#plt.title('Average Time per prompt (s)')
plt.grid(True, axis='y', linestyle='--', alpha=0.7)
plt.grid(True, axis='x', linestyle='--', alpha=0.7)
plt.yticks(fontsize=14)                           
plt.xticks(rotation=45, ha='right', fontsize=14)
plt.tight_layout()
plt.show()

**prediction interval per repetition diagram**

In [ ]:
import numpy as np
import scipy.stats as stats
def prediction_interval(samples, confidence=0.95):
    mean = np.mean(samples)
    std_dev = np.std(samples, ddof=1)
    n = len(samples)
    # if n > 2:
    #     t_crit = stats.t.ppf((1 - confidence) / 2, df=n - 1)
    #     std_dev = np.std(samples, ddof=1)
    # else:
    #     t_crit = 12.706
    #     variance = sum((x - mean) ** 2 for x in samples) / (len(samples) - 1)  # sample variance
    #     std_dev = variance ** 0.5
    margin_of_error = t_crit * std_dev * np.sqrt(2 / n)
    print(std_dev)
    lower_bound = (mean - margin_of_error) if margin_of_error > 0 else (mean + margin_of_error)
    upper_bound = (mean + margin_of_error) if margin_of_error > 0 else (mean - margin_of_error)
    return lower_bound, upper_bound, margin_of_error
def prediction_interval_incremental(scores):
    for n in range(2, len(scores)+1):
        mean = np.mean(scores[:n])
        l, u, err = prediction_interval(scores[:n])
        u = min(100, u)
        print(f"n={n} : {l:.2f} {u:.2f} {mean:.2f} +- {abs(err):.2f}") # {(u-l):.2f}") 

In [ ]:
#for grok-3-mini default
scores = [88.99,	88.75,	89.64,	88.43,	87.69,	91.38,	85.36,	83.66,	87.9]

prediction_interval_incremental(scores)

In [ ]:
#for gemini-2.5-flash
scores = [96.98,	95.47,	95.75,	98.22,	96.3]

prediction_interval_incremental(scores)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load Excel file
df1 = pd.read_excel(
    "/kaggle/input/all-accuracies-scatter-plot-v5/all_accuracies_v2.xlsx",
    sheet_name="gemini-2.5-flash_pi_over_repeat",
    index_col=0
)

# Ensure column names are treated as strings or numbers
df1.columns = df1.columns.astype(str)

# Create a single plot
fig, ax = plt.subplots(figsize=(4, 4))

# Use actual column names as x-axis values
x_vals = df1.columns.astype(float)  # Convert to numeric if needed

# Plot each row safely
for label in df1.index:
    y_vals = df1.loc[label]
    valid_vals = y_vals.dropna()
    x_clean = valid_vals.index.astype(float)
    y_clean = valid_vals.values
    ax.plot(x_clean, y_clean, marker='o', label=label, color='#0080ff')

ax.set_xticks(x_vals)
ax.set_ylim(80, 105)

# Left-side label
ax.text(
    x=-0.15,
    y=0.5,
    s='gemini-2.5-flash',
    rotation=90,
    fontsize=12,
    va='center',
    ha='center',
    transform=ax.transAxes
)

# Top-right stat label
ax.text(
    0.95, 0.95, '93.83 ± 2.56, n=3',
    fontsize=12, ha='right', va='top',
    transform=ax.transAxes
)

ax.set_title("default")
ax.grid(True)

plt.tight_layout()
plt.show()


**HEATMAP CALCULATION V2**

In [12]:
from openpyxl import load_workbook, Workbook
import pandas as pd

def calculate_jaccard(cell_value):
    if not cell_value or '|' not in cell_value:
        return None

    parts = cell_value.split('|')
    if len(parts) != 2:
        return None

    set1 = {v.strip() for v in parts[0].split(',') if v.strip()}
    set2 = {v.strip() for v in parts[1].split(',') if v.strip()}

    union = set1 | set2
    if not union:
        return 1.0  # Perfect match if both are empty

    intersection = set1 & set2
    return round(len(intersection) / len(union), 4)

def average_jaccard_across_sheets(file_path, sheet_names, output_path):
    # Load workbook and sheets
    wb = load_workbook(file_path, data_only=True)
    sheets = [wb[name] for name in sheet_names]

    # Create output workbook and sheet
    out_wb = Workbook()
    out_ws = out_wb.active
    out_ws.title = "Average Jaccard"

    max_rows = max(sheet.max_row for sheet in sheets)
    max_cols = max(sheet.max_column for sheet in sheets)

    for row in range(1, max_rows + 1):
        for col in range(1, max_cols + 1):
            jaccards = []
            for sheet in sheets:
                cell = sheet.cell(row=row, column=col)
                jaccard = calculate_jaccard(cell.value)
                if jaccard is not None:
                    jaccards.append(jaccard)
            if jaccards:
                avg_jaccard = round(sum(jaccards) / len(jaccards), 4)
                out_ws.cell(row=row, column=col, value=avg_jaccard)

    out_wb.save(output_path)
    print(f"Averaged Jaccard values written to '{output_path}'")

# === USAGE ===
# Define input and output paths
input_excel = "/kaggle/input/grok-3-mini-def-all-3-results/grok-3-mini_def_all_3_results.xlsx"
output_excel = "/kaggle/working/average_jaccard_all_sheets.xlsx"

# Define the sheet names to process
sheet_names = ["Φύλλο1", "Φύλλο2", "Φύλλο3"]  # Replace with your actual sheet names

# Run the function
average_jaccard_across_sheets(input_excel, sheet_names, output_excel)


Averaged Jaccard values written to '/kaggle/working/average_jaccard_all_sheets.xlsx'
